In [1]:
# Add the parent directory to Python path so we can import from core
import sys
import os
# Get the current working directory and go up one level to the backend directory
backend_dir = os.path.dirname(os.getcwd())
if backend_dir not in sys.path:
    sys.path.append(backend_dir)

In [ ]:
import asyncio, datetime, uuid
from core.issue import IssueContext
from core.llm import LLMClient

llm_client = LLMClient()
issue = IssueContext(llm_client)

issue.start()

# Let background tasks spin up without blocking the event loop
await asyncio.sleep(1)

# send a message to the issue
issue.ingest({
    "id": str(uuid.uuid4()),
    "type": "issue.begin",
    "payload": {
        "id": "test_1",
        "name": "Issue Description",
        "description": "This is the issue description initially given by the user.",
        "rationale": "",
        "outcomes": {},
        "result": "My truck revs up when I put the keys in the ignition but it won't start."
    },
    "issue_id": issue.id,
    "meta": {
        "ts": datetime.datetime.utcnow().isoformat(),
        "source": "system"
    }
})

# Keep the notebook cell alive briefly to stream prints from tasks
await asyncio.sleep(600)

await issue.stop()



Executing handler for event type: issue.begin


C:\Users\jakej\AppData\Local\Temp\ipykernel_6092\3192048691.py:27: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.datetime.utcnow().isoformat(),


We have initial scenario: issue description: truck revs up when keys in ignition but won't start. Need to produce initial updated_probabilities: we have no prior hypotheses. We must generate plausible issues. Use test result: revs up but won't start suggests battery, alternator, starter, fuel, ignition, engine mechanical. So hypotheses: battery failure, faulty starter solenoid, bad spark plugs, clogged fuel filter, failed fuel pump, ignition coil failure, ECU issue, immobilizer, etc.

We need to update probabilities based on test result. Since it's first test, we consider initial scores based on how often seen? Use prior probabilities? We'll assign baseline 1.0 for each.

But rule: Update probabilities using multiplicative scoring rule based ONLY on most recent test. For each hypothesis, apply factor: need to determine support/contradiction. Test result: revs up but won't start. That supports battery? Wait battery is the cause of no start: battery dead? But revs up means engine cranks?

CancelledError: 

 need to output JSON with fields test_text, test_instructions, etc. The test description: attach fuel pressure gauge to rail and crank to measure. So instructions: step 1: locate fuel rail. step 2: attach gauge. step 3: start engine. step 4: record pressure. Also safety warnings: don't get near hot engine, keep gauge stable, etc. Output result field: "Fuel pressure (psi)" type number.

So produce JSON: "test_text": maybe "Fuel Pressure Test". "test_instructions": array of step objects with step_number and step_text.

We can include test_result_field_label: "Fuel pressure (psi)". test_result_field_type: "number". test_result_field_options: none. safety_and_warnings: array.

We might leave test_result_field_options blank. So JSON.

Return only JSON.